# Импорт модулей

In [ ]:
!pip install tensorflow==1.15.0

In [ ]:
tf.__version__

In [ ]:
import keras
keras.__version__

In [ ]:
# Датасет с изображениями
# https://drive.google.com/open?id=1XHDr44w65JHCceXZzTbYX6p3pDfEmLfx

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Dropout, Reshape, Input, Lambda
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from os import listdir
import os
import shutil
import PIL
from IPython.display import display
from PIL import Image
import pathlib
import csv
!pip install patool
import patoolib
import base64

# Подготовка данных

In [ ]:
!ls data_documents

In [ ]:
arr = image.load_img('data_documents/licenses/i (29).jpeg')
display(arr)

In [ ]:
print('Всего счетов: ',  len(listdir('data_documents/bills')))
print('Всего договоров: ',  len(listdir('data_documents/contracts')))
print('Всего лицензий: ',  len(listdir('data_documents/licenses')))

In [ ]:
%%bash
#mkdir train
#mkdir test

In [ ]:
%%bash
mkdir train/bills
mkdir train/contracts
mkdir train/licenses

In [ ]:
%%bash
mkdir test/bills
mkdir test/contracts
mkdir test/licenses

In [ ]:
!ls

In [ ]:
coef_train = 0.75
#coef_test = 0.3
bills_value = len(listdir('data_documents/bills'))
contracts_value = len(listdir('data_documents/contracts'))
licenses_value =  len(listdir('data_documents/licenses'))

In [ ]:
def copy_images(path, coef, class_name):
  image_value = len(listdir(path))
  file_names = listdir(path)
  
  # 75 % изображений перемещаем в папку train/ 'class_name'
  for i in range(0, int(coef*image_value)):
    shutil.copy(os.path.join(path, file_names[i]), os.path.join('train', class_name))

  # 25 % изображений перемещаем в папку test/ 'class_name'
  for i in range(int(coef*image_value), int(image_value)):
    shutil.copy(os.path.join(path, file_names[i]), os.path.join('test', class_name))


In [ ]:
copy_images('data_documents/bills', coef_train, 'bills')

In [ ]:
copy_images('data_documents/contracts', coef_train, 'contracts')

In [ ]:
copy_images('data_documents/licenses', coef_train, 'licenses')

In [ ]:
print('Счета: ', len(listdir('train/bills')),len(listdir('test/bills')))

In [ ]:
print('Договора: ', len(listdir('train/contracts')),len(listdir('test/contracts')))

In [ ]:
print('Лицензии: ', len(listdir('train/licenses')),len(listdir('test/licenses')))

In [ ]:
img_path = 'train/bills/i (97).jpeg'
img = image.load_img(img_path, target_size=(224, 224))
plt.imshow(img)
plt.show()

In [ ]:
# Размеры изображения
img_width, img_height = 224, 224
# Размерности тензоров
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 32

# Модель без генератора

## Загрузка данных

In [ ]:
# Загружаем данные
def data_load(path):
  X = []
  y = []
  class_names = ['bills', 'contracts', 'licenses']
  for class_name in class_names:
    path_temp = os.path.join(path, class_name)
    file_names = listdir(path_temp)
    for file in file_names:
      if 'jpeg' in file:
        img = image.load_img(os.path.join(path_temp, file), target_size=(224, 224))
        # convert to numpy array
        img_array = image.img_to_array(img)
        X.append(img_array)
        y.append(class_names.index(class_name))
  return X, y

In [ ]:
X_train, y_train = data_load('train')

In [ ]:
X_test, y_test = data_load('test')

In [ ]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

In [ ]:
# convert back to image
img_pil = image.array_to_img(X_test[100])
plt.imshow(img_pil)
plt.show()

In [ ]:
y_test[100]

In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# OHE

In [ ]:
X_train = np.asarray(X_train)/255
X_test = np.asarray(X_test)/255
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

## Строим сеть

In [ ]:
#Sequential
vgg16_NN = VGG16(weights='imagenet', 
                  include_top=False, 
                  input_shape=input_shape)
model = Sequential()
model.add(vgg16_NN)
#model.add(Flatten())
model.add(Dense(256, activation='relu'))
#model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Reshape((7*7*64,)))
#model.add(Dropout(0.25))
model.add(Dense(3, activation='softmax'))

In [ ]:
lr = 1e-4
model.compile(
              loss='categorical_crossentropy',
              optimizer=Adam(lr=lr),
              metrics=['accuracy']
              )

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
import time
start_all_time = time.time()
history = model.fit(
                      x=X_train, y = y_train,
                      batch_size = 32,
                      validation_split=0.2,
                      epochs=4)
print (f'Обучилась за: {time.time() - start_all_time}')

In [ ]:
plt.plot(history.history['acc'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_acc'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
y_predict = np.argmax(y_predict, axis=1)

In [ ]:
y_test = np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_predict, y_test)

# Сохраняем модель SaveModel

In [ ]:
# export the model
import keras.backend as K

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter
import os, shutil
import numpy as np
from keras.utils.data_utils import get_file

models_dir = os.path.join(os.getcwd(), "models")
model_name = "document"
model_version = "00001"
if os.path.exists(models_dir):
  shutil.rmtree(models_dir)
export_path = os.path.join(models_dir, model_name, model_version)
print(export_path)
if os.path.exists(export_path):
  shutil.rmtree(export_path)
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'images': model.input},
                                 outputs={'scores': model.output})

with K.get_session() as sess:
  builder.add_meta_graph_and_variables(sess=sess,
                                      tags=[tag_constants.SERVING],
                                      signature_def_map={'predict': signature})
  builder.save()

In [ ]:
# list the files and directories in our model export directory
os.listdir(export_path)

In [ ]:
os.path.join(os.getcwd(), "/images/bill.jpg")

# Запросы к серверу

In [ ]:
import tensorflow as tf
from skimage import io

# Select an image that the model has not seen during training
image = '/Users/dimon/Desktop/Школа Больших Данных/Кейс_классификация_документов/tensorflow_serving_example_v2/images/contract.jpeg'
print(image)
# Display the image for the user
io.imshow(image)
io.show()

# Preprocessing function that defines a TensorFlow graph to preprocess the image
# to a format that the model needs as its input
def preprocess_image(image_buffer):
  """Preprocess JPEG encoded bytes to 3D float Tensor."""

  # Decode the string as an RGB JPEG.
  # Note that the resulting image contains an unknown height and width
  # that is set dynamically by decode_jpeg. In other words, the height
  # and width of image is unknown at compile-time.
  image = tf.image.decode_jpeg(image_buffer, channels=3)
  # After this point, all image pixels reside in [0,1).
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  # Crop the central region of the image with an area containing 87.5% of
  # the original image.
  image = tf.image.central_crop(image, central_fraction=0.875)
  # Resize the image to the height and width needed for VGG-16 model 
  # Here it's 224, but it can vary based on the architecture of the model used.
  image = tf.expand_dims(image, 0)
  image = tf.image.resize_bilinear(
      image, [224, 224], align_corners=False)
  image = tf.squeeze(image)
  return image

In [ ]:
from grpc.beta import implementations

from tensorflow.core.framework import types_pb2
from tensorflow.python.platform import flags
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2
import requests

# Specify the port on which TensorFlow Model Server is running in Part 2
#server = "0.0.0.0:8501"
server = "localhost:8501"
# Gather host and port data for the model server
host, port = server.split(':')

SERVER_URL = 'http://localhost:8501/v1/models/document:predict'

with open(image, 'rb') as f:
# Create a channel
  channel = implementations.insecure_channel(host, int(port))
# Create a prediction service (gRPC)
# We can then send gRPC requests using Protocol Buffers (protobuf) to this service 
# and get back a response as a protobuf which we can then extract information from.
# See prediction_service.proto in TensorFlow Serving code base
# for gRPC request/response details
  stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)
  # Read the image data from a JPEG image
  data = f.read()  
  # Preprocess the data using the TensorFlow graph defined above in
  # `preprocess_image()` function
  im_data = preprocess_image(data)
  with tf.Session() as sess:
    # We need to run the session in order to compute the values in the 
    # graph defined in `preprocess_image()` function
    sess.run(im_data)
    # Convert the tensor returned from the preoprocess_image() function to a
    # numpy array using `eval()` function
    im_data_arr = im_data.eval()
  print(type(im_data_arr), im_data_arr.shape)
  payload = { 'signature_name': 'predict', "instances": [{'images': im_data_arr.tolist()}] }

  r = requests.post(SERVER_URL, json=payload)
  print(r.text)